In [1]:
import os
import json
import pandas as pd
import openai
from dotenv import load_dotenv

In [2]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [3]:
# Load the environment variables from .env file
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
cache_file = '../cache_summary.json'

# Load cache from disk if it exists
try:
    with open(cache_file, 'r') as f:
        cache = json.load(f)
except FileNotFoundError:
    cache = {}

In [5]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate

llm = init_chat_model("gpt-4.1-nano-2025-04-14", model_provider="openai", temperature=0)

In [6]:
df = pd.read_json("../data/2023_processed.json")
df = df.dropna(subset=["maintext"])
print(df.language.unique()) # Articles always in English
print(df.date_publish.map(lambda x: x[:4]).unique()) # Articles always published in 2023

['en']
['2023']


In [7]:
cols_to_keep = ["date_publish", "description", "maintext", "title", "url", "related_companies"]

In [8]:
df.head()

,authors,date_download,date_modify,date_publish,description,filename,image_url,language,localpath,maintext,source_domain,title,title_page,title_rss,url,mentioned_companies,related_companies,industries,named_entities,prev_day_price_C,next_day_price_C,curr_day_price_C,sentiment,emotion,news_outlet,prev_day_price_BA,next_day_price_BA,curr_day_price_BA,prev_day_price_GOOGL,next_day_price_GOOGL,curr_day_price_GOOGL,prev_day_price_MA,next_day_price_MA,curr_day_price_MA,prev_day_price_BABA,next_day_price_BABA,curr_day_price_BABA,prev_day_price_WFC,next_day_price_WFC,curr_day_price_WFC,prev_day_price_TSLA,next_day_price_TSLA,curr_day_price_TSLA,prev_day_price_T,next_day_price_T,curr_day_price_T,prev_day_price_GS,next_day_price_GS,curr_day_price_GS,prev_day_price_WMT,next_day_price_WMT,curr_day_price_WMT,prev_day_price_MSFT,next_day_price_MSFT,curr_day_price_MSFT,prev_day_price_JPM,next_day_price_JPM,curr_day_price_JPM,prev_day_price_V,next_day_price_V,curr_day_price_V,prev_day_price_BAC,next_day_price_BAC,curr_day_price_BAC,prev_day_price_AVGO,next_day_price_AVGO,curr_day_price_AVGO,prev_day_price_INTC,next_day_price_INTC,curr_day_price_INTC,prev_day_price_NVDA,next_day_price_NVDA,curr_day_price_NVDA,prev_day_price_MU,next_day_price_MU,curr_day_price_MU,prev_day_price_AMZN,next_day_price_AMZN,curr_day_price_AMZN,prev_day_price_MRK,next_day_price_MRK,curr_day_price_MRK,prev_day_price_COST,next_day_price_COST,curr_day_price_COST,prev_day_price_AAPL,next_day_price_AAPL,curr_day_price_AAPL,prev_day_price_CVX,next_day_price_CVX,curr_day_price_CVX,prev_day_price_PG,next_day_price_PG,curr_day_price_PG,prev_day_price_ADBE,next_day_price_ADBE,curr_day_price_ADBE,prev_day_price_PYPL,next_day_price_PYPL,curr_day_price_PYPL,prev_day_price_KO,next_day_price_KO,curr_day_price_KO,prev_day_price_GE,next_day_price_GE,curr_day_price_GE,prev_day_price_ORCL,next_day_price_ORCL,curr_day_price_ORCL,prev_day_price_DIS,next_day_price_DIS,curr_day_price_DIS,prev_day_price_CSCO,next_day_price_CSCO,curr_day_price_CSCO,prev_day_price_BRK,next_day_price_BRK,curr_day_price_BRK,prev_day_price_NFLX,next_day_price_NFLX,curr_day_price_NFLX,prev_day_price_CRM,next_day_price_CRM,curr_day_price_CRM,prev_day_price_LLY,next_day_price_LLY,curr_day_price_LLY,prev_day_price_HD,next_day_price_HD,curr_day_price_HD,prev_day_price_CMCSA,next_day_price_CMCSA,curr_day_price_CMCSA,prev_day_price_SHOP,next_day_price_SHOP,curr_day_price_SHOP,prev_day_price_ROKU,next_day_price_ROKU,curr_day_price_ROKU,prev_day_price_QCOM,next_day_price_QCOM,curr_day_price_QCOM,prev_day_price_PFE,next_day_price_PFE,curr_day_price_PFE,prev_day_price_JNJ,next_day_price_JNJ,curr_day_price_JNJ,prev_day_price_UNH,next_day_price_UNH,curr_day_price_UNH,prev_day_price_NIO,next_day_price_NIO,curr_day_price_NIO,prev_day_price_XOM,next_day_price_XOM,curr_day_price_XOM,prev_day_price_VZ,next_day_price_VZ,curr_day_price_VZ,prev_day_price_MRNA,next_day_price_MRNA,curr_day_price_MRNA,prev_day_price_SQ,next_day_price_SQ,curr_day_price_SQ
0,[HAVEN DALEY],2023-06-23 07:21:46+00:00,NaN,2023-06-23 05:38:00,"At Tyler Malek's ice cream parlors, one cook's...",https%3A%2F%2Ffinance.yahoo.com%2Fnews%2Fus-to...,https://s.yimg.com/ny/api/res/1.2/Gwc7ViNgtrTh...,en,NaN,"LOS GATOS, Calif. (AP) — At Tyler Malek's ice ...",finance.yahoo.com,The US has tons of leftover food. Upcycling se...,NaN,NaN,https://finance.yahoo.com/news/us-tons-leftove...,[C],"[BSAC, FHN, PACW, BSMX, VLY, MBRG, SMMF, GNBC,...",[6021],"[{'entity_group': 'LOC', 'word': 'Calif', 'nor...",46.63,46.24,46.02,"{'negative': 0.00045554022653900006, 'neutral'...","{'neutral': 0.714272022247314, 'disgust': 0.24...",finance.yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [9]:
df = df[cols_to_keep]
df.head()

,date_publish,description,maintext,title,url,related_companies
0,2023-06-23 05:38:00,"At Tyler Malek's ice cream parlors, one cook's...","LOS GATOS, Calif. (AP) — At Tyler Malek's ice ...",The US has tons of leftover food. Upcycling se...,https://finance.yahoo.com/news/us-tons-leftove...,"[BSAC, FHN, PACW, BSMX, VLY, MBRG, SMMF, GNBC,..."
1,2023-08-26 14:00:17,"The worst result, after buying shares in a com...","The worst result, after buying shares in a com...",Baker Hughes (NASDAQ:BKR) shareholders have ea...,https://finance.yahoo.com/news/baker-hughes-na...,"[CHU, INSG, S, TDS, DCM, TMUS, CHT, SPOK, VEON..."
2,2023-12-06 16:57:28,(Bloomberg) -- An insolvency filing by Signa H...,(Bloomberg) -- An insolvency filing by Signa H...,Signa’s Insolvency Yields Long List of Credito...,https://finance.yahoo.com/news/signa-insolvenc...,[TXT]
3,2023-06-14 07:21:56,Swiss citizens vote this weekend on whether to...,By John Revill\nZURICH (Reuters) - Swiss citiz...,Low-tax Switzerland votes on global minimum co...,https://finance.yahoo.com/news/low-tax-switzer...,"[IGLD, RAMP, NSR, TWTR, ACXM, COR, PINS, META,..."
4,2023-01-10 20:23:00,Nationally recognized branding agency HAVEN Cr...,"WAXHAW, N.C., Jan. 10, 2023 /PRNewswire/ -- Na...",National Branding Agency HAVEN Creative Looks ...,https://finance.yahoo.com/news/national-brandi...,"[FIS, FRXB, AAQC, EEX, AUXO, BBOX, GHY, CTLP, ..."


In [10]:
df["maintext"].sample(1).values[0].split("\n")

['CHENNAI (Reuters) - A decision by Taiwan\'s Foxconn to withdraw from a $19.5 billion semiconductor joint venture with Vedanta is a "credit negative" for the Indian conglomerate\'s UK parent Vedanta Resources, debt research firm CreditSights said on Thursday.',
 'Earlier this week, Foxconn said it had withdrawn from the JV, dumping a deal they had signed last year to set up semiconductor and display production plants in India. Vedanta on Wednesday said it would still enter the space this year having lined up partners.',
 'CreditSights had previously expected a "minimal credit impact" on Vedanta Resources based on the old arrangement, as Volcan, Vedanta\'s holding entity was undertaking the semiconductor investments.',
 '"Since the semiconductor venture will be now parked directly under Vedanta, we see a higher probability that a good portion of the project funding will come from Vedanta," CreditSights, part of the Fitch Group, wrote in a note.',
 "CreditSights also expects further str

In [11]:
prompt_template = PromptTemplate.from_template("""
As a professional summarizer for a financial newspaper, create a concise and comprehensive summary of the provided article while adhering to these guidelines:

Craft a summary that is detailed, thorough, in-depth, and complex, while maintaining clarity and conciseness.

Incorporate main ideas and essential information, eliminating extraneous language and focusing on critical aspects.

Rely strictly on the provided text, without including external information.

Format the summary in paragraph form for easy understanding.
                                               
You are creating the summary to be put in the frontpage of the newspaper, so be catchy and critical.

Use from 250 to 400 words.                                                                                         

By following this optimized prompt, you will generate an effective summary that encapsulates the essence of the given article in a clear, concise, and reader-friendly manner.

Article: {Article}
""")

In [12]:
llm.invoke(prompt_template.format(Article=df["maintext"].values[0]))

AIMessage(content='In a compelling shift toward sustainability, the upcycling movement is transforming the food industry by turning food waste into high-quality, innovative products, exemplified by Salt & Straw’s pioneering ice cream flavors. Led by head ice cream maker Tyler Malek, the chain utilizes remnants such as whey from yogurt production and brewing byproducts like rice and grains to craft unique flavors like lemon curd and chocolate barley milk, challenging traditional notions of waste and emphasizing the importance of resourcefulness. This approach aligns with a broader consumer trend demanding transparency about food origins and environmental impact, as evidenced by the rising popularity of upcycled ingredients in grocery stores and restaurants nationwide. The Upcycled Food Association reports that over 35 million tons of food are wasted annually in the U.S., representing roughly 40% of the nation’s food production and costing more than $200 billion, highlighting the urgent 

In [13]:
df_limited = df.iloc[:50, :]

In [14]:
def get_summary_with_cache(x):
    if x in cache:
        print("Using cached summary")
        return cache[x]
    summary = llm.invoke(prompt_template.format(Article=x)).content
    cache[x] = summary
    with open(cache_file, 'w') as f:
        json.dump(cache, f)
    return summary

In [15]:
df_limited["summary"] = df_limited["maintext"].map(lambda x: get_summary_with_cache(x))

Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached 

/tmp/ipykernel_112426/812272221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limited["summary"] = df_limited["maintext"].map(lambda x: get_summary_with_cache(x))


In [16]:
df_limited.head()

,date_publish,description,maintext,title,url,related_companies,summary
0,2023-06-23 05:38:00,"At Tyler Malek's ice cream parlors, one cook's...","LOS GATOS, Calif. (AP) — At Tyler Malek's ice ...",The US has tons of leftover food. Upcycling se...,https://finance.yahoo.com/news/us-tons-leftove...,"[BSAC, FHN, PACW, BSMX, VLY, MBRG, SMMF, GNBC,...","In a compelling shift toward sustainability, t..."
1,2023-08-26 14:00:17,"The worst result, after buying shares in a com...","The worst result, after buying shares in a com...",Baker Hughes (NASDAQ:BKR) shareholders have ea...,https://finance.yahoo.com/news/baker-hughes-na...,"[CHU, INSG, S, TDS, DCM, TMUS, CHT, SPOK, VEON...",Baker Hughes Company (NASDAQ:BKR) has demonstr...
2,2023-12-06 16:57:28,(Bloomberg) -- An insolvency filing by Signa H...,(Bloomberg) -- An insolvency filing by Signa H...,Signa’s Insolvency Yields Long List of Credito...,https://finance.yahoo.com/news/signa-insolvenc...,[TXT],"Signa Holding, the umbrella entity of Rene Ben..."
3,2023-06-14 07:21:56,Swiss citizens vote this weekend on whether to...,By John Revill\nZURICH (Reuters) - Swiss citiz...,Low-tax Switzerland votes on global minimum co...,https://finance.yahoo.com/news/low-tax-switzer...,"[IGLD, RAMP, NSR, TWTR, ACXM, COR, PINS, META,...",Switzerland faces a pivotal referendum this we...
4,2023-01-10 20:23:00,Nationally recognized branding agency HAVEN Cr...,"WAXHAW, N.C., Jan. 10, 2023 /PRNewswire/ -- Na...",National Branding Agency HAVEN Creative Looks ...,https://finance.yahoo.com/news/national-brandi...,"[FIS, FRXB, AAQC, EEX, AUXO, BBOX, GHY, CTLP, ...","Haven Creative, a nationally acclaimed brandin..."


In [17]:
df_limited.iloc[2, :].summary

'Signa Holding, the umbrella entity of Rene Benko’s sprawling property and retail empire, has filed for insolvency, shedding light on its intricate and high-stakes financial web. The court documents, obtained by Bloomberg, reveal a creditor list comprising 273 entities, including major financial institutions like BNP Paribas and Julius Baer, sovereign wealth funds such as Saudi Arabia’s Public Investment Fund, and a diverse array of smaller creditors ranging from private security firms to local service providers. Notably, the filing exposes the empire’s exposure to trophy assets like New York’s Chrysler Building, London’s Selfridges, and Berlin’s KaDeWe, which are now at risk amid mounting financial distress.\n\nThe insolvency estimate signals potential losses of approximately €5 billion ($5.4 billion) in a liquidation scenario, yet critical questions remain unresolved—particularly the origins of billions in contingent liabilities and the true extent of debt linked to off-balance-sheet

In [18]:
len(df_limited.iloc[2, :].description.split())

73

In [19]:
len(df_limited.iloc[2, :].maintext.split())

831

In [20]:
len(df_limited.iloc[2, :].summary.split())

394

In [21]:
import asyncio

In [22]:
async def process_single_article(article_text, prompt_template):
    """Process a single article asynchronously"""
    if article_text in cache:
        print("Using cached summary")
        return cache[article_text]
    try:
        formatted_prompt = prompt_template.format(Article=article_text)
        response = await llm.ainvoke(formatted_prompt)
        summary = response.content
        cache[article_text] = summary
        with open(cache_file, 'w') as f:
            json.dump(cache, f)
        return summary
    except Exception as e:
        print(f"Error processing article: {e}")
        return None

async def process_articles_batch(df, prompt_template, batch_size=5):
    """Process articles in batches to avoid rate limits"""
    results = []
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        
        # Create tasks for the batch
        tasks = [
            process_single_article(row["maintext"], prompt_template) 
            for _, row in batch.iterrows()
        ]
        
        # Process batch concurrently
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        results.extend(batch_results)
        
        # Optional: Add delay between batches to respect rate limits
        if i + batch_size < len(df):
            await asyncio.sleep(1)  # 1 second delay between batches
    
    return results

In [23]:
res = await process_articles_batch(df.iloc[:500, :], prompt_template)

Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary


Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached summary
Using cached 

In [24]:
res[0]

'In a compelling shift toward sustainability, the upcycling movement is transforming the food industry by turning food waste into high-quality, innovative products, exemplified by Salt & Straw’s pioneering ice cream flavors. Led by head ice cream maker Tyler Malek, the chain utilizes remnants such as whey from yogurt production and rice and grain residues from brewing to craft unique flavors like lemon curd and chocolate barley milk, challenging traditional notions of waste. This approach aligns with a broader consumer trend demanding transparency about food origins and environmental impact, as over 35 million tons of food—approximately 40% of U.S. food production—are discarded annually, costing the economy over $200 billion. The Upcycled Food Association (UFA) plays a crucial role in mainstreaming this movement, certifying products with its “Upcycling Certified” seal, which now adorns 450 items, up from just 30 in 2021. This certification highlights efforts to combat archaic cosmetic 

In [25]:
#res = await process_articles_batch(df, prompt_template)

In [26]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [27]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="articles",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [28]:
from uuid import uuid4
from langchain_core.documents import Document

In [29]:
df.head()

,date_publish,description,maintext,title,url,related_companies
0,2023-06-23 05:38:00,"At Tyler Malek's ice cream parlors, one cook's...","LOS GATOS, Calif. (AP) — At Tyler Malek's ice ...",The US has tons of leftover food. Upcycling se...,https://finance.yahoo.com/news/us-tons-leftove...,"[BSAC, FHN, PACW, BSMX, VLY, MBRG, SMMF, GNBC,..."
1,2023-08-26 14:00:17,"The worst result, after buying shares in a com...","The worst result, after buying shares in a com...",Baker Hughes (NASDAQ:BKR) shareholders have ea...,https://finance.yahoo.com/news/baker-hughes-na...,"[CHU, INSG, S, TDS, DCM, TMUS, CHT, SPOK, VEON..."
2,2023-12-06 16:57:28,(Bloomberg) -- An insolvency filing by Signa H...,(Bloomberg) -- An insolvency filing by Signa H...,Signa’s Insolvency Yields Long List of Credito...,https://finance.yahoo.com/news/signa-insolvenc...,[TXT]
3,2023-06-14 07:21:56,Swiss citizens vote this weekend on whether to...,By John Revill\nZURICH (Reuters) - Swiss citiz...,Low-tax Switzerland votes on global minimum co...,https://finance.yahoo.com/news/low-tax-switzer...,"[IGLD, RAMP, NSR, TWTR, ACXM, COR, PINS, META,..."
4,2023-01-10 20:23:00,Nationally recognized branding agency HAVEN Cr...,"WAXHAW, N.C., Jan. 10, 2023 /PRNewswire/ -- Na...",National Branding Agency HAVEN Creative Looks ...,https://finance.yahoo.com/news/national-brandi...,"[FIS, FRXB, AAQC, EEX, AUXO, BBOX, GHY, CTLP, ..."


In [30]:
documents = [Document(page_content=s, metadata={"source_id": i, "date_publish": d}) for i, (s, d) \
             in enumerate(df[["title", "date_publish"]].values)]

In [31]:
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)

In [32]:
query = "News about Apple Iphone shipments"
results = vectorstore.similarity_search(query, k=10)

In [33]:
results

[Document(id='040b7384-92bb-4454-be3b-f821cce44935', metadata={'date_publish': '2023-07-24 10:56:33', 'source_id': 23407}, page_content='Apple aims to keep iPhone shipments steady - Bloomberg News'),
 Document(id='f9501764-406c-4bf8-89a3-ac8abc1467a0', metadata={'source_id': 6648, 'date_publish': '2023-07-24 14:01:42'}, page_content='Apple Aims to Keep iPhone Shipments Steady Despite 2023 Turmoil'),
 Document(id='6a1c4a15-39a1-41a3-8899-abd33741e3bd', metadata={'source_id': 3919, 'date_publish': '2023-01-27 09:00:48'}, page_content='Apple’s iPhone Dominated China Last Quarter Despite Disruptions'),
 Document(id='6ca629a0-cf7c-4f17-90cd-00eb06010587', metadata={'date_publish': '2023-12-15 21:25:38', 'source_id': 13722}, page_content="China's ban on Apple's iPhone accelerates- Bloomberg News"),
 Document(id='1a03cb44-ddd6-4520-b6ad-c9393c15d5e6', metadata={'source_id': 8629, 'date_publish': '2023-08-03 21:06:00'}, page_content='Apple’s Sluggish iPhone Sales Overshadow Services Growth'),


In [34]:
from langchain.chains import RetrievalQA

In [35]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [36]:
query = "what is the situation of the Iphone shipment in 2023?"
qa_chain.invoke(query)

{'query': 'what is the situation of the Iphone shipment in 2023?',
 'result': "In 2023, Apple aimed to keep iPhone shipments steady despite various market challenges. While there have been disruptions, such as China's ban on Apple's iPhone and some demand fluctuations, Apple has continued to dominate the Chinese market and maintain its global presence. Notably, iPhone exports from India have doubled, surpassing $2.5 billion, indicating growth in that region. Additionally, some iPhone models, like the iPhone 15 Pro Max, experienced delivery delays, suggesting supply chain or demand adjustments. Overall, despite a challenging environment, Apple has managed to sustain its iPhone shipments and market share in 2023.",
 'source_documents': [Document(id='f9501764-406c-4bf8-89a3-ac8abc1467a0', metadata={'source_id': 6648, 'date_publish': '2023-07-24 14:01:42'}, page_content='Apple Aims to Keep iPhone Shipments Steady Despite 2023 Turmoil'),
  Document(id='040b7384-92bb-4454-be3b-f821cce44935',

In [37]:
vector_store = Chroma(
    collection_name="articles",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

documents = [Document(page_content=s, metadata={"source_id": i, "date_publish": d}) for i, (s, d) \
             in enumerate(df[["description", "date_publish"]].values)]

vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)

In [38]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

qa_chain.invoke(query)

{'query': 'what is the situation of the Iphone shipment in 2023?',
 'result': 'The situation of iPhone shipments in 2023 is that Apple aims to keep the shipments steady despite a projected decline in the overall smartphone market. They are targeting around 85 million units, roughly in line with the previous year. Apple is also considering raising the prices for Pro models to support this goal. Additionally, demand for the new iPhone 15 models appears to be high, as indicated by longer delivery times compared to the previous year.',
 'source_documents': [Document(id='f9501764-406c-4bf8-89a3-ac8abc1467a0', metadata={'date_publish': '2023-07-24 14:01:42', 'source_id': 6648}, page_content='Apple Aims to Keep iPhone Shipments Steady Despite 2023 Turmoil'),
  Document(id='040b7384-92bb-4454-be3b-f821cce44935', metadata={'source_id': 23407, 'date_publish': '2023-07-24 10:56:33'}, page_content='Apple aims to keep iPhone shipments steady - Bloomberg News'),
  Document(id='b3164f87-706f-4f33-8e9

In [40]:
query="How Meta is leveraging the AI revolution?"
qa_chain.invoke(query)

{'query': 'How Meta is leveraging the AI revolution?',
 'result': "Meta is heavily investing in artificial intelligence by developing more powerful AI systems and integrating AI into its products. The company has dedicated a new team focused on AI products as part of its 'year of efficiency' initiative. Meta aims to use AI to increase the return on investment (ROI) for advertising, especially in response to challenges like Apple's ad tracking policies. Additionally, Meta is showcasing new AI tools and developing advanced AI systems to stay competitive with other tech giants like Google and ChatGPT. The company is also actively involved in discussions about AI risks and has invested significantly in defensive and protective systems to manage misinformation and disinformation, particularly around elections. Overall, Meta's strategy involves leveraging AI to enhance advertising, improve user experience, and ensure safety and security on its platforms.",
 'source_documents': [Document(id='